# Mid Semester Project - DEEP Reinforcements Learning
### Part 2: REINFROCE Algorithm

Student: Jonathan Mendelson 308564293


##### Imports

In [1]:
# gym
import gymnasium
import minigrid
from minigrid.wrappers import RGBImgObsWrapper, RGBImgPartialObsWrapper, ImgObsWrapper, FullyObsWrapper, RGBImgPartialObsWrapper, ReseedWrapper, ActionBonus
import imageio

# general utils
import matplotlib.pyplot as plt
import os
import time

# torch
import torch
from torchsummary import summary

# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# my imports
from src.display_utils import embed_mp4, launch_tb
from src.eval_utils import record_agent_video
from src.cnn_policy import CNNPolicyNetwork
from src.reinforce import ReinforceTrainer


# notebook setup
%load_ext autoreload
%autoreload 2

#### Set experiment name:

In [2]:
# set experiment name
experiment_name = 'RF_02'

# create log dir
os.makedirs(f".\\experiments\\{experiment_name}", exist_ok=True)
timestamp = time.strftime("%b-%d_%H-%M-%S")
# timestamp = 'Feb-28_16-14-34' # TODO temp
log_path = os.path.join('experiments', experiment_name, timestamp, 'logs')
# create content dir
content_path = os.path.join('experiments', experiment_name, timestamp, 'content')
os.makedirs(content_path, exist_ok=True)
# set models dir
model_path = os.path.join('experiments', experiment_name, timestamp,'models')
os.makedirs(model_path, exist_ok=True)

#### Environment Initialization
Define general env params and build initial env with two rooms.

In [3]:
# env options
env_names = ["MiniGrid-MultiRoom-N2-S4-v0", "MiniGrid-MultiRoom-N4-S5-v0","MiniGrid-MultiRoom-N6-v0"] 

# env config
highlight = False
render_mode = "rgb_array"

In [4]:
# build env_0 with two rooms
env_2_rooms = gymnasium.make(env_names[0], render_mode=render_mode, highlight=highlight)
env_2_rooms = ImgObsWrapper(RGBImgPartialObsWrapper(env_2_rooms)) 

#### Agent Initialization
We will actually use the same model as the AC model, except we will only utilize the actor part (and not the critic).

In [13]:
poilcy = CNNPolicyNetwork(device, env_2_rooms.action_space.n)
summary(poilcy, (3,56,56))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 55, 55]             208
              ReLU-2           [-1, 16, 55, 55]               0
         MaxPool2d-3           [-1, 16, 27, 27]               0
            Conv2d-4           [-1, 32, 12, 12]           8,224
              ReLU-5           [-1, 32, 12, 12]               0
            Conv2d-6             [-1, 64, 9, 9]          32,832
              ReLU-7             [-1, 64, 9, 9]               0
           Flatten-8                 [-1, 5184]               0
            Linear-9                  [-1, 128]         663,680
           Linear-10                    [-1, 7]             903
Total params: 705,847
Trainable params: 705,847
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.04
Forward/backward pass size (MB): 1.02
Params size (MB): 2.69
Estimated T

#### Initialize Trainer

In [6]:
# logger
# tb_process, tb_writer = launch_tb(log_path)
# tb_process.communicate()

In [14]:
# trainer
trainer = ReinforceTrainer(device, policy_network=poilcy, lr=3e-4, gamma = 0.99, writer = None)

#### Training Schedule
We will train the agent according to a training schedule with a few parts:
* Two rooms with max steps of 1000, for 10K episodes.
* Two rooms with max steps of 20, for 10K episodes.
* Four rooms with max steps of 2000, for 10K episodes.



TODO finish



##### Two Rooms, 10 Envs:

In [8]:
# trainer.policy_network.load_state_dict(torch.load('experiments\RF_02\RF_2Room_100_2.pt'))

In [32]:
experiment_phase = '2Room_1k'
trainer.update_phase(experiment_phase)
trainer.set_lr = 3e-4

# remove the steps limit from the env
max_steps = 100
env_2_rooms.unwrapped.max_steps = max_steps

In [33]:
trainer.train(env = env_2_rooms, num_episodes = 2500, max_steps = max_steps, entropy_weight = 1e-3)

episode: 10, last episode reward: 0.757, average_reward: 0.585, last episode length: 26, average length: 45.143, entropy: 0.592
episode: 20, last episode reward: 0.766, average_reward: 0.684, last episode length: 25, average length: 33.471, entropy: 0.636
episode: 30, last episode reward: 0.253, average_reward: 0.589, last episode length: 82, average length: 43.815, entropy: 0.667
episode: 40, last episode reward: 0, average_reward: 0.501, last episode length: 99, average length: 52.595, entropy: 0.664
episode: 50, last episode reward: 0, average_reward: 0.499, last episode length: 99, average length: 52.511, entropy: 0.644
episode: 60, last episode reward: 0.964, average_reward: 0.521, last episode length: 3, average length: 50.035, entropy: 0.641
episode: 70, last episode reward: 0.757, average_reward: 0.518, last episode length: 26, average length: 50.254, entropy: 0.643
episode: 80, last episode reward: 0.793, average_reward: 0.518, last episode length: 22, average length: 50.519, 

In [40]:
video_path = f"{content_path}\\{experiment_phase}_3.mp4"
vid = record_agent_video(device, poilcy, env_2_rooms, video_path)
embed_mp4((video_path))

In [36]:
torch.save(poilcy.state_dict(), 'RF_2Room_100_5000.pt')

TODO: validation step

##### Four rooms, 2000 steps: 

In [ ]:
trainer.policy_network.load_state_dict(torch.load('experiments\RF_02\RF_2Room_100_5000.pt'))

In [46]:
experiment_phase = '4Room'
# trainer
trainer.update_phase(experiment_phase)
trainer.set_lr(new_lr = 3e-4)

# next env
env_4_rooms = gymnasium.make(env_names[1], render_mode=render_mode, highlight=highlight)
env_4_rooms = ImgObsWrapper(RGBImgPartialObsWrapper(env_4_rooms)) 

# remove the steps limit from the env
max_steps = 2000
env_4_rooms.unwrapped.max_steps = max_steps

In [47]:
trainer.train(env = env_4_rooms, num_episodes = 10000, max_steps = max_steps, entropy_weight = 1e-3)

KeyboardInterrupt: 

In [45]:
video_path = f"{content_path}\\{experiment_phase}_2.mp4"
vid = record_agent_video(device, trainer.policy_network, env_4_rooms, video_path)
embed_mp4((video_path))

In [ ]:
torch.save(policy.state_dict(), '4Room_100.pt')

##### Four rooms, 1 env:

In [ ]:
env_4 = gymnasium.make(env_names[1], render_mode=render_mode, highlight=highlight, max_steps=100)
env_partial_rgb_4 = ImgObsWrapper(RGBImgPartialObsWrapper(env_4))
env_partial_rgb_seed_4 = ReseedWrapper(env_partial_rgb_4, seeds = range(1), seed_idx = 0)
# env_partial_rgb_seed_4_bonus = ActionBonus(env_partial_rgb_seed_4)

In [ ]:
optimizer = Adam(AC_model.parameters(), lr=1e-4, weight_decay=0)
a2c_trainer = Trainer(device = device, model = AC_model, optimizer=optimizer,
                    writer = tb_writer, model_path = model_path)

In [ ]:
experiment_phase = '4Room_1'
l_params = {'gamma':0.99, 'actor_weight': 0.5, 'critic_weight': 0.5, 'entropy_weight': -0.01, 'reward_amplify': 10}
a2c_trainer.A2CTrain(env = env_partial_rgb_seed_4, l_params = l_params, rollout_size = 20, save_every_episodes = 2000,
                    episodes = 10000, val_every_episodes = 50, experiment_phase = experiment_phase)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
video_path = f"{content_path}\\{experiment_phase}_1.mp4"
vid = record_agent_video(device, AC_model, env_partial_rgb_seed_4, video_path)
embed_mp4((vid))

In [ ]:
torch.save(AC_model.state_dict(), '4Room_100.pt')

In [ ]:
# env.close()
tb_process.kill()